In [86]:
import csv
import pandas as pd
import re
import sqlite3
from sqlite3 import Error
import ast 
import math
import os
import numpy as np

In [87]:
conn = sqlite3.connect("receipts.db")
cursor = conn.cursor()

## SQL Questions Answer:

### 1): What are the top 5 brands by receipts scanned for most recent month?

Filtered brand code is none value, current output is null because data is not in most recent year/month

In [88]:
cursor.execute("""
select i.brand_code as brand_code, count(r.receipt_id) as receipt_counts
from (
select *
from receipt
where strftime('%Y-%m', scanned_date) = (strftime('%Y-%m', date('now'))) ) r 
left join receipt_item ri
on r.receipt_id = ri.receipt_id
left join item i
on ri.barcode = i.barcode
where i.brand_code != 'None'
group by i.brand_code
order by count(r.receipt_id) desc
limit 5
""")
answer = cursor.fetchall()
answer

[]

### 2): How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [89]:
cursor.execute("""
select brand_code, dense_rank() over (order by receipt_counts desc) as rank_num
from

(select i.brand_code as brand_code, count(r.receipt_id) as receipt_counts
from (
select *
from receipt
where strftime('%Y-%m', scanned_date) = (strftime('%Y-%m',  date('now', 'start of month', '-1 month'))) ) r 
left join receipt_item ri
on r.receipt_id = ri.receipt_id
left join item i
on ri.barcode = i.barcode
where i.brand_code != 'None'
group by i.brand_code
) 

where brand_code in 

(select i.brand_code as brand_code
from (
select *
from receipt
where strftime('%Y-%m', scanned_date) = (strftime('%Y-%m', date('now'))) ) r 
left join receipt_item ri
on r.receipt_id = ri.receipt_id
left join item i
on ri.barcode = i.barcode
where i.brand_code != 'None'
group by i.brand_code
order by count(r.receipt_id) desc
limit 5)
""")
answer = cursor.fetchall()
answer

[]

### 3): When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

current data only show rejected because no "accepted" string in 'rewardsreceiptstatus'

In [90]:
cursor.execute("""
select rs.receipt_status_name as status, avg(total_spent) as average_spend
from receipt r
left join receipt_status rs
on r.receipt_status_id = rs.receipt_status_id
where rs.receipt_status_name in ('ACCEPTED','REJECTED')
group by rs.receipt_status_name
""")
answer = cursor.fetchall()
answer

[('REJECTED', 23.326056338028184)]

### 5): Which brand has the most spend among users who were created within the past 6 months?

Filtered brand code is none value, current output is null because data range is old

In [96]:
cursor.execute("""
select i.brand_code, sum(i.final_price)
from receipt r
left join user u
on r.user_id = u.user_id
left join receipt_item ri
on r.receipt_id = ri.receipt_id
left join item i
on ri.barcode = i.barcode
where i.brand_code != 'None'
and strftime('%Y-%m', u.created_date) >= (strftime('%Y-%m',  date('now', 'start of month', '-6 month')))
group by i.brand_code
order by sum(i.final_price) desc
limit 1
""")
answer = cursor.fetchall()
answer

[]